In [1]:
import pandas as pd
import random as rd
import datetime as dt
import pickle,glob,re

In [2]:
cumule=True
repNom="/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/"
repNomSortie="/Users/gilles/Box Sync/2015-Data/FlexionAdjectifs/"
repNomSortie="/Users/gilles/Box Sync/2015-Data/TEMP/"

In [3]:
listeTirages=glob.glob(repNom+"*-Tirage.pkl")

In [4]:
def tiragesNom(nom):
    result=[tirage for tirage in listeTirages if nom in tirage]
    return result

In [5]:
compositionLongitudinale=[("1Ko",10),("10Ko",9),("100Ko",9),("1Mo",19)]
compositionLongitudinale=[("10Mo",20)]
compositionLongitudinale=[("10Ko",10),("100Ko",9),("1Mo",9)]

In [6]:
serieLongitudinale=[]
for serie,nbSerie in compositionLongitudinale:
    for nom in rd.sample(tiragesNom(serie),nbSerie):
        m=re.match(ur"^.*-%s-(\d{2})-T.*-F.*-Tirage.*"%serie,nom)
        if m:
#            print m.group(1), nom
            serieLongitudinale.append(nom)
        else:
            print "mauvais nom de fichier",nom
serieLongitudinale

['/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/MGC-10Ko-09-T10000-F2994-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/MGC-10Ko-03-T10000-F2991-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/MGC-10Ko-08-T10000-F2985-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/MGC-10Ko-01-T10000-F3102-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/MGC-10Ko-07-T10000-F3000-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/MGC-10Ko-05-T10000-F3044-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/MGC-10Ko-06-T10000-F3013-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/MGC-10Ko-02-T10000-F3019-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/MGC-10Ko-04-T10000-F3107-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/MGC-10Ko-00-T10000-F3068-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/MGC-100Ko-04-T100000-F10012-Tirage.pk

In [7]:
def lireLexique(nomLexique):
    with open(nomLexique, 'rb') as input:
        lexique=pickle.load(input)
    return lexique

In [8]:
cumulLexique=lireLexique(serieLongitudinale[0])
cumulLexique["tir1"]=0

In [9]:
timeStamp=dt.datetime.now().strftime("%y%m%d-%H%M")
for nTirage,tirage in enumerate(serieLongitudinale):
    lexique=lireLexique(tirage)
    if cumule:
        cumulLexique["tir1"]=cumulLexique["tir1"]+lexique["tir1"]
    else:
        cumulLexique["tir1"]=lexique["tir1"]
    lexique1=cumulLexique[cumulLexique["tir1"]>0]
    nbTokens=lexique1["tir1"].sum()
    nbTypes=lexique1["tir1"].count()
    print "(%d,%d),"%(nbTokens,nbTypes)
    sortieNom=repNomSortie+"Longitudinal-%02d-T%d-F%d-%s.pkl"%(nTirage,nbTokens,nbTypes,timeStamp)
    with open(sortieNom,"wb") as output:
        pickle.dump(lexique1, output, pickle.HIGHEST_PROTOCOL)

(10000,2994),
(20000,4568),
(30000,5701),
(40000,6652),
(50000,7385),
(60000,8040),
(70000,8568),
(80000,9081),
(90000,9561),
(100000,9974),
(200000,12719),
(300000,14300),
(400000,15326),
(500000,16117),
(600000,16725),
(700000,17209),
(800000,17602),
(900000,17925),
(1000000,18210),
(2000000,19654),
(3000000,20160),
(4000000,20440),
(5000000,20590),
(6000000,20701),
(7000000,20791),
(8000000,20865),
(9000000,20927),
(10000000,20973),
